# JSON-RPC Basics
### **Request** Structure
```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "some.method",
    "params":{...}
}
```

### **Response** Structure
*Success*
```json
{
    "jsonrpc": "2.0",
    "id":1,
    "result": {...}
}
```
*Failure*
```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "error": {
        "code": 123,
        "message": "Something broke"
    }
}
```
### Fields
**Required**
| Field              | Meaning                                    |
| ------------------ | ------------------------------------------ |
| `"jsonrpc": "2.0"` | Must always be exactly `"2.0"`             |
| `"method"`         | The name of the API method                 |
| `"id"`             | A unique request ID (any type except null) |

**Optional**
| Field      | Meaning                                  |
| ---------- | ---------------------------------------- |
| `"params"` | Arguments to the method (object or list) |

### Batch Requests
```json
[
  { "jsonrpc": "2.0", "id": 1, "method": "math.add", "params": [1,2] },
  { "jsonrpc": "2.0", "id": 2, "method": "math.mul", "params": [3,4] }
]
```

### Notifications
when you send a request without `id`, it becomes a notification or request without response
```json
{
  "jsonrpc": "2.0",
  "method": "metrics/ping"
}
```
Used for events like "didOpen", "statusChanged" etc. MCP uses these for server→client updates.

### JSON-RPC over different transports
**Over HTTP**:  
you `post` the JSON to a URL  
  
**Over WebSockets/TCP**:  
Just send the JSON message as strings  
  
**Over SSE (for MCP)**:  
- Client connects to `/sse`
- Server sents messages as SSE events
- Client writes the JSON-RPC messages over the *write channel* (usually HTTP POST or WebSocket)


In [22]:
!docker ps

CONTAINER ID   IMAGE                COMMAND                  CREATED         STATUS         PORTS                                         NAMES
81d7804abdf6   docker/mcp-gateway   "/docker-mcp gateway…"   8 seconds ago   Up 7 seconds   0.0.0.0:8811->8811/tcp, [::]:8811->8811/tcp   docker_mcp_host-gateway-1


# Important
MCP needs to carry out an **Initialization Phase**
1. Client sends initilization request
2. Server sends a response
3. Client needs to notify the server

Takaways:
1. Initialize the client with these headers:
    ```json
    {
        "Mcp-Protocol-Version": "2024-11-05",
        "Accept": "application/json, text/event-stream",
    }
    ```
2. You need the *same async client* for the *same request sequence* or you would need to begin a new **Initialization Phase**
3. When the server responds you need to get the **mcp-session-id**
4. Subsequent requests needs the extra header: `headers={"Mcp-Session-Id": mcp-session-id}`
5. Since, `--transport=streaming`, MPC server return SSE or `f"data: {some_data}\n\n"`. So return `response.text` and parse the sse output

### Initialization jRPC request:
```python
payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "test-client",
                "version": "1.0.0",
            },
        },
    }
```
### Notification jRPC request:
It will not recieve any response from the server
```python
payload = {
        "jsonrpc": "2.0",
        "method": "notifications/initialized",
    }
```
### Tool List jRPC request:
```python
payload = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/list",
        "params": {},
    }
```

In [ ]:
# Cannot run this on ipynb
import httpx
import json
import asyncio

server_url = "http://localhost:8811/mcp"

def parse_sse_response(response_text: str):
    lines = response_text.split("\n")
    for line in lines:
        if line.startswith("data: "):
            data = line[6:]
            try:
                return json.loads(data)
            except json.JSONDecodeError:
                print(f"Could not parse JSON: {data}")
    return None

async def initialize_session(client: httpx.AsyncClient):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "test-client",
                "version": "1.0.0",
            },
        },
    }

    resp = await client.post(server_url, json=payload)
    resp.raise_for_status()
    session_id = resp.headers.get("Mcp-Session-Id") or resp.headers.get("mcp-session-id")
    print("Initialize response headers:")
    print(dict(resp.headers))
    print("Mcp-Session-Id:", session_id) # Get this session for subsequent request

    return resp.text, session_id

async def send_initialized_notification(client: httpx.AsyncClient, session_id: str):
    payload = {
        "jsonrpc": "2.0",
        "method": "notifications/initialized",
    }

    resp = await client.post(
        server_url,
        json=payload,
        headers={"Mcp-Session-Id": session_id}, # Adding the session id as a header
    )
    resp.raise_for_status()
    return resp.text

async def get_tools_list(client: httpx.AsyncClient, session_id: str):
    payload = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/list",
        "params": {},
    }

    resp = await client.post(
        server_url,
        json=payload,
        headers={"Mcp-Session-Id": session_id}, # Adding the session id as a header
    )
    resp.raise_for_status()
    return resp.text

async def main():
    try:
        async with httpx.AsyncClient(
            timeout=300,
            headers={
                # Need this headers
                "Mcp-Protocol-Version": "2024-11-05",
                "Accept": "application/json, text/event-stream",
            },
            limits=httpx.Limits(max_keepalive_connections=1, max_connections=1),
        ) as client:
            print("Initializing session...")
            init_response, session_id = await initialize_session(client)
            print("Initialization response:")
            print(init_response)

            init_data = parse_sse_response(init_response)
            if init_data and "result" in init_data:
                print("\n===PARSED INITIALIZATION===")
                print(json.dumps(init_data["result"], indent=2))

            print("\nSending initialized notification...")
            notif_response = await send_initialized_notification(client, session_id)
            print("Notification response:")
            # repr prints all the characters like '\n' etc. Here the response will usually be '' since notification jRPC does not have a response
            print(repr(notif_response))

            print("\nGetting tools list...")
            tools_response = await get_tools_list(client, session_id)
            print("Tools list raw response:")
            print(tools_response)

            tools_data = parse_sse_response(tools_response)
            if tools_data:
                print("\n===PARSED TOOLS LIST===")
                print(json.dumps(tools_data, indent=2))

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    asyncio.run(main())


# Output:
```json
{
  "jsonrpc": "2.0",
  "id": 2,
  "result": {
    "tools": [
      {
        "_meta": {
          "_fastmcp": {
            "tags": []
          }
        },
        "description": "Extract key facts from a Wikipedia article, optionally focused on a topic.",
        "inputSchema": {
          "properties": {
            "count": {
              "default": 5,
              "type": "integer"
            },
            "title": {
              "type": "string"
            },
            "topic_within_article": {
              "default": "",
              "type": "string"
            }
          },
          "required": [
            "title"
          ],
          "type": "object"
        },
        "name": "extract_key_facts",
        "outputSchema": {
          "additionalProperties": true,
          "type": "object"
        }
      },
      {
        "_meta": {
          "_fastmcp": {
            "tags": []
          }
        },
        "description": "Get the full content of a Wikipedia article.",
        "inputSchema": {
          "properties": {
            "title": {
              "type": "string"
            }
          },
          "required": [
            "title"
          ],
          "type": "object"
        },
        "name": "get_article",
        "outputSchema": {
          "additionalProperties": true,
          "type": "object"
        }
      },
      {
        "_meta": {
          "_fastmcp": {
            "tags": []
          }
        },
        "description": "Get the coordinates of a Wikipedia article.",
        "inputSchema": {
          "properties": {
            "title": {
              "type": "string"
            }
          },
          "required": [
            "title"
          ],
          "type": "object"
        },
```